In [4]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np

In [5]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

In [6]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [7]:
train_ds = TensorDataset(inputs,targets)

In [8]:
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [9]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle = True)

In [10]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 92.,  87.,  64.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 74.,  66.,  43.],
        [102.,  43.,  37.]])
tensor([[ 82., 100.],
        [103., 119.],
        [ 56.,  70.],
        [ 57.,  69.],
        [ 22.,  37.]])


In [11]:
model = nn.Linear(3,2)
print(f"Model.Weights: {model.weight}")
print(f"Model.Bias: {model.bias}")

Model.Weights: Parameter containing:
tensor([[ 0.2617,  0.5282,  0.2739],
        [ 0.4507, -0.1173, -0.5206]], requires_grad=True)
Model.Bias: Parameter containing:
tensor([ 0.4770, -0.5369], requires_grad=True)


In [12]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2617,  0.5282,  0.2739],
         [ 0.4507, -0.1173, -0.5206]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4770, -0.5369], requires_grad=True)]

In [13]:
pred = model(inputs)
pred

tensor([[ 66.7461,   2.1158],
        [ 88.2998,  -3.1682],
        [109.9048,  -7.2452],
        [ 60.0171,  21.1248],
        [ 88.4104, -17.1450],
        [ 66.4796,   2.6838],
        [ 88.0456,  -3.5714],
        [110.4404,  -7.3150],
        [ 60.2835,  20.5568],
        [ 88.4226, -18.1163],
        [ 66.4918,   1.7126],
        [ 88.0334,  -2.6002],
        [110.1591,  -6.8419],
        [ 60.0049,  22.0960],
        [ 88.6768, -17.7130]], grad_fn=<AddmmBackward0>)

In [14]:
loss_fn = F.mse_loss

In [15]:
loss = loss_fn(model(inputs),targets)

In [16]:
loss

tensor(5589.3657, grad_fn=<MseLossBackward0>)

In [17]:
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

In [18]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    for epoch in range(1, num_epochs+1):
        for xb,yb in train_dl:
            pred  = model(xb)
            loss = loss_fn(pred,yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        if epoch%10 == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item()}")
            

In [19]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 734.6847534179688
Epoch [20/100], Loss: 488.45281982421875
Epoch [30/100], Loss: 272.984130859375
Epoch [40/100], Loss: 84.90351104736328
Epoch [50/100], Loss: 9.145484924316406
Epoch [60/100], Loss: 227.40896606445312
Epoch [70/100], Loss: 130.11648559570312
Epoch [80/100], Loss: 140.37380981445312
Epoch [90/100], Loss: 86.94795227050781
Epoch [100/100], Loss: 52.77508544921875


In [20]:
preds = model(inputs)
print(preds)

tensor([[ 58.1065,  72.2888],
        [ 80.5360,  95.9453],
        [119.7534, 139.7826],
        [ 26.8348,  49.0177],
        [ 95.8385, 103.8615],
        [ 56.9607,  71.3454],
        [ 80.0628,  95.1022],
        [119.9008, 139.9409],
        [ 27.9806,  49.9611],
        [ 96.5111, 103.9618],
        [ 57.6333,  71.4457],
        [ 79.3902,  95.0019],
        [120.2267, 140.6257],
        [ 26.1622,  48.9175],
        [ 96.9843, 104.8049]], grad_fn=<AddmmBackward0>)


In [21]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [22]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.4585, 68.5866]], grad_fn=<AddmmBackward0>)